In [1]:
import pandas as pd
import numpy as np
import sentencepiece as sp
from torch.utils.data import Dataset

In [2]:
data = pd.read_csv(".data/benchmarkingGS_v1-0_similarityMeasure_sequence_v3-1.csv")
data[data['trainTest'] == 'train']

,uniprotID_A,uniprotID_B,isInteraction,trainTest,RNAseqHPA,tissueHPA,tissueCellHPA,subcellularLocationHPA,bioProcessUniprot,cellCompUniprot,molFuncUniprot,domainUniprot,motifUniprot,Bgee,sequence_A,sequence_B
1,O00161,P56962,1,train,0.825131,0.851690,0.675880,0.000000,0.190693,0.200000,0.353553,0.0,0.0,0.922975,MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLD...,MSEDEEKVKLRRLEPAIQKFIKIVIPTDLERLRKHQINIEKYQRCR...
2,P82979,Q01081,1,train,0.930790,0.954869,0.911887,0.000000,0.547723,0.365148,0.408248,0.0,0.0,NaN,MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHA...,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...
3,O60678,Q14524,1,train,0.219384,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,NaN,MCSLASGATGGRGAVENEEDLPELSDSGDEAAWEDEDDADLPHGKQ...,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,P10275,Q15648,1,train,0.500558,-0.311704,-0.371061,0.000000,0.169811,0.375000,0.157895,0.0,0.0,NaN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MKAQGETEESEKLSKMSSLLERLHAKFNQNRPWSETIKLVRQVMEK...
5,P05413,P05556,1,train,0.290361,0.340155,0.549298,0.666667,0.000000,0.096225,0.000000,0.0,0.0,0.790815,MVDAFLGTWKLVDSKNFDDYMKSLGVGFATRQVASMTKPTTIIEKN...,MNLQPIFWIGLISSVCCVFAQTDENRCLKANAKSCGECIQAGPNCG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131555,P41732,Q8NBD8,0,train,0.619041,-0.160794,-0.238177,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.727030,MASRRMETKPVITCLKTLLIIYSFVFWITGVILLAVGVWGKLTLGT...,MASAEPLTALSRWYLYAIHGYFCEVMFTAAWEFVVNLNWKFPGVTS...
131556,Q96JN2,Q96PV4,0,train,0.655477,0.841197,0.651238,NaN,0.000000,0.000000,0.000000,0.0,0.0,-0.049936,MQAMEGEVLLPALYEEEEEEEEEEEEVEEEEEQVQKGGSVGSLSVN...,MALTLLEDWCKGMDMDPRKALLIVGIPMECSEVEIQDTVKAGLQPL...
131557,O43169,Q9BYR5,0,train,0.071429,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,-0.657258,MSGSMATAEASGSDGKGQEVETSVTYYRLEEVAKRNSLKELWLVIH...,MVNSCCGSVCSDQGCGLENCCRPSCCQTTCCRTTCCRPSCCVSSCC...
131558,Q04864,Q8WW22,0,train,0.527243,NaN,NaN,NaN,0.000000,0.316228,0.000000,0.0,0.0,NaN,MASGAYNPYIEIIEQPRQRGMRFRYKCEGRSAGSIPGEHSTDNNRT...,MVKETQYYDILGVKPSASPEEIKKAYRKLALKYHPDKNPDEGEKFK...


In [6]:
data.describe

<bound method NDFrame.describe of        uniprotID_A uniprotID_B  isInteraction trainTest  RNAseqHPA  tissueHPA  \
0           P28223      P41595              1     test2   0.160188  -0.449930   
1           O00161      P56962              1     train   0.825131   0.851690   
2           P82979      Q01081              1     train   0.930790   0.954869   
3           O60678      Q14524              1     train   0.219384        NaN   
4           P10275      Q15648              1     train   0.500558  -0.311704   
...            ...         ...            ...       ...        ...        ...   
268494      O95678      Q8IYX3              0     test2   0.022646        NaN   
268495      O95835      Q8WUJ0              0     test2   0.854290        NaN   
268496      P60409      Q9NUB4              0     test2   0.894159        NaN   
268497      O43294      Q6ZRT6              0     test2   0.062636        NaN   
268498      P36405      P36639              0     test2   0.566853   0.9398

In [7]:
data["sequence_A"].iloc[1]

'MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWTVDSENRTNLSCEGCLSPSCLSLLHLQEKNWSALLTAVVIILTIAGNILVIMAVSLEKKLQNATNYFLMSLAIADMLLGFLVMPVSMLTILYGYRWPLPSKLCAVWIYLDVLFSTASIMHLCAISLDRYVAIQNPIHHSRFNSRTKAFLKIIAVWTISVGISMPIPVFGLQDDSKVFKEGSCLLADDNFVLIGSFVSFFIPLTIMVITYFLTIKSLQKEATLCVSDLGTRAKLASFSFLPQSSLSSEKLFQRSIHREPGSYTGRRTMQSISNEQKACKVLGIVFFLFVVMWCPFFITNIMAVICKESCNEDVIGALLNVFVWIGYLSSAVNPLVYTLFNKTYRSAFSRYIQCQYKENKKPLQLILVNTIPALAYKSSQLQMGQKKNSKQDAKTTDNDCSMVALGKQHSEEASKDNSDGVNEKVSCV'

In [2]:
metrics = pd.read_csv("lightning_logs/version_238/metrics.csv")

In [3]:
metrics

,acc,apr,auroc,epoch,step,train_loss,train_loss_step,val_loss
0,0.6375,NaN,NaN,0,49,NaN,0.685881,NaN
1,0.5250,NaN,NaN,0,99,NaN,0.681449,NaN
2,0.5625,NaN,NaN,0,149,NaN,0.682561,NaN
3,0.6000,NaN,NaN,0,199,NaN,0.671231,NaN
4,0.5000,NaN,NaN,0,249,NaN,0.715683,NaN
...,...,...,...,...,...,...,...,...
165,0.6750,NaN,NaN,3,7999,NaN,0.626401,NaN
166,0.8000,NaN,NaN,3,8049,NaN,0.542160,NaN
167,0.7125,NaN,NaN,3,8099,NaN,0.574199,NaN
168,0.7125,NaN,NaN,3,8149,NaN,0.572982,NaN


In [4]:
metrics[metrics['train_loss']]

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=170)] are in the [columns]"

# Prepare Dataset

First RAPPPID creates list of integers for given AS-Sequence

In [12]:
def get_aa_code(aa):
    # Codes based on IUPAC-IUB
    # https://web.expasy.org/docs/userman.html#AA_codes

    aas = ['PAD', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'O', 'U']
    wobble_aas = {
        'B': ['D', 'N'],
        'Z': ['Q', 'E'],
        'X': ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
    }

    if aa in aas:
        return aas.index(aa)

    elif aa in ['B', 'Z', 'X']:
        # Wobble
        idx = randint(0, len(wobble_aas[aa])-1)
        return aas.index(wobble_aas[aa][idx])

def encode_seq(seq):
    return [get_aa_code(aa) for aa in seq]

In [13]:
# input
protein = 'MRPSGTAGAALLALL'

In [14]:
# outout
token1 = encode_seq(protein)
token1

[13, 2, 15, 16, 8, 17, 1, 8, 1, 1, 11, 11, 1, 11, 11]

These list of integers is then saved

## Example SentencePieceTokenizer

Converts sequence string into list of integers

In [15]:
import sentencepiece as sp
tokenizer_file = 'scripts/rapppid/spm.model'
tokenizer = sp.SentencePieceProcessor(model_file=tokenizer_file)

In [17]:
protein

'MRPSGTAGAALLALL'

In [24]:
tokens = np.array(tokenizer.encode(protein, enable_sampling=True, alpha=0.1, nbest_size=-1))
tokens

array([ 96,  22,  23,   9,  19, 112,  64,   5,   7,   7,  29,   7])

# Lightning Dataset

tokenizes given AS-Sequence (String) 


In [4]:
class RapppidDataset(Dataset):
    def __init__(self, rows, seqs, model_file, trunc_len=1000, vocab_size=2000):
        super().__init__()
        	
        self.rows = rows
        self.seqs = seqs
        self.trunc_len = trunc_len

        self.spp = sp.SentencePieceProcessor(model_file=model_file)

    @staticmethod
    def static_encode(trunc_len: int, spp, seq: str, sp: bool = True, pad: bool = True):
        aas = ['PAD', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'O', 'U']

        toks = "".join([aas[r] for r in seq[:trunc_len]])
        
        if sp:
                toks = np.array(spp.encode(toks, enable_sampling=True, alpha=0.1, nbest_size=-1))
        if pad:
             pad_len = trunc_len - len(toks)
             toks = np.pad(toks, (0, pad_len), 'constant')

        return toks

    def encode(self, seq: str, sp: bool = True, pad: bool = True):

        aas = ['PAD', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'O', 'U']

        toks = "".join([aas[r] for r in seq[:self.trunc_len]])
        
        if sp:
                toks = np.array(self.spp.encode(toks, enable_sampling=True, alpha=0.1, nbest_size=-1))
        if pad:
             pad_len = self.trunc_len - len(toks)
             toks = np.pad(toks, (0, pad_len), 'constant')

        return toks

    def __getitem__(self, idx):

        p1, p2, label = self.rows[idx]

        p1_seq = self.encode(self.seqs[p1], sp=True, pad=True)

        p2_seq = self.encode(self.seqs[p2], sp=True, pad=True)

        p1_seq = torch.tensor(p1_seq).long()
        p2_seq = torch.tensor(p2_seq).long()
        label = torch.tensor(label).long()

        return (p1_seq, p2_seq, label)

    def __len__(self):
        return len(self.rows)

In [48]:
spp = sp.SentencePieceProcessor(model_file='scripts/rapppid/spm.model')

In [55]:
 # method within Datset class
 def encode(seq: str, sp: bool = True, pad: bool = False):

        # aas = ['PAD', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'O', 'U']

        # toks = "".join([aas[r] for r in seq])
        # print(toks)

        toks = seq
        
        if sp:
                toks = np.array(spp.encode(toks, enable_sampling=True, alpha=0.1, nbest_size=-1))
        if pad:
             pad_len = trunc_len - len(toks)
             toks = np.pad(toks, (0, pad_len), 'constant')

        return toks

In [53]:
amino_acid

'MRPSGTAGAALLALL'

In [56]:
encode(seq = amino_acid)

array([248,   4,  22,  49,  19, 112,  64,  29,   7,   5,  24])

In [52]:
encode(seq = amino_acid)

MRPSGTAGAALLALL


array([248,   4,  22,  49, 100,   5,  64,   5,   7,   7,   5,   7,   7])